# V3 Fraction Collector (Runze Valve)

In [1]:
from fraction_collector_v4 import FractionCollector
from sample_producer_flowreactor import SampleProducer
import threading
import queue
import time

In [2]:
# Initialize the FractionCollector
collector = FractionCollector(sensor_id=1, runze_valve_port='COM9', runze_valve_address=0, runze_valve_num_port=10, collection_num=3, waste_num=6)

Connected to CNC Machine!
Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']


In [3]:
# Define well locations
loc = 'well_plate_location'
#loc = '96_well_plate_location'

In [4]:
# Shared queue and event
sample_queue = queue.Queue()
stop_event = threading.Event()
number_of_samples = 1

# Initialize producer
producer = SampleProducer(
    reaction_profile_csv='HCW4.csv',
    sample_queue=sample_queue,
    stop_event=stop_event,
    tubing_length_m=1.0,
    tubing_id_inch=0.03,
    buffer_factor=1.2
)
producer.start()

[16:26:39] === Sample Collection Summary ===
[16:26:39] Reaction name         : HCW4
[16:26:39] Waste time (s)        : 1582.9
[16:26:39] Collection time (s)   : 540.7
[16:26:39] Flow rate (mL/min)    : 0.50
[16:26:39] Tubing length (m)     : 1.0
[16:26:39] Tubing ID (inch)      : 0.03
[16:26:39] Buffer factor         : 1.2
[16:26:39] Tubing delay (s)      : 54.7
[16:26:39] Adjusted waste time   : 1648.5 s
[16:26:39] Adjusted collect time : 450.6 s
[16:26:39] =================================
[16:26:39] Starting waste phase for 1648.5 seconds...


[16:27:09] Wasting... 30 / 1648 seconds elapsed
[16:27:39] Wasting... 60 / 1648 seconds elapsed
[16:28:09] Wasting... 90 / 1648 seconds elapsed
[16:28:39] Wasting... 120 / 1648 seconds elapsed
[16:29:09] Wasting... 150 / 1648 seconds elapsed
[16:29:39] Wasting... 180 / 1648 seconds elapsed
[16:30:09] Wasting... 210 / 1648 seconds elapsed
[16:30:39] Wasting... 240 / 1648 seconds elapsed
[16:31:09] Wasting... 270 / 1648 seconds elapsed
[16:31:39] Wasting... 300 / 1648 seconds elapsed
[16:32:09] Wasting... 330 / 1648 seconds elapsed
[16:32:39] Wasting... 360 / 1648 seconds elapsed
[16:33:09] Wasting... 390 / 1648 seconds elapsed
[16:33:39] Wasting... 420 / 1648 seconds elapsed
[16:34:09] Wasting... 450 / 1648 seconds elapsed
[16:34:39] Wasting... 480 / 1648 seconds elapsed
[16:35:09] Wasting... 510 / 1648 seconds elapsed
[16:35:39] Wasting... 540 / 1648 seconds elapsed
[16:36:09] Wasting... 570 / 1648 seconds elapsed
[16:36:39] Wasting... 600 / 1648 seconds elapsed
[16:37:09] Wasting... 6

In [ ]:
# Simulate collection
loc_index = 0
loc_threshold = loc_index + number_of_samples
collection_thread = None

while not stop_event.is_set():
    try:
        sample = sample_queue.get(timeout=0.1)
        if collection_thread and collection_thread.is_alive():
            print("❌ Not enough time between samples — shutting down.")
            stop_event.set()
            break

        #collection_thread = threading.Thread(target=collect_sample_sim, args=(sample, loc_index, 3))
        collection_thread = threading.Thread(target=collector.collect_fraction, args=(70, loc, loc_index, 20)) #real collection
        collection_thread.start()
        loc_index += 1

    except queue.Empty:
        pass

    if loc_index >= loc_threshold:  # match number of samples
        break

# Wait for last collection to finish
if collection_thread and collection_thread.is_alive():
    collection_thread.join()

# Final result
if stop_event.is_set() or loc_index < loc_threshold:
    print("❌ Not enough samples")
else:
    print("✅ All samples collected successfully.")

Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']
Rinsing collection tubing for 20 drops...

Current port set to 3
Drop Counter (Drops) Started
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Current port set to 6
Rinsing complete.

Moving to location: well_plate_location at index 0
Movement commands rendered: 4
Moved safely To (X35.5, Y129.5, Z-12):  ['ok']

Current port set to 3
Starting fraction collection at well_plate_location (index 0)...

Drop Counter (Drops) Started
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Fraction collection of well_plate_location (index 0) complete.

Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']
✅ All samples collected successfully.


In [5]:
# Simulate collection
loc_index = 0
loc_threshold = loc_index + number_of_samples*3
collection_thread = None

while not stop_event.is_set():
    try:
        sample = sample_queue.get(timeout=0.1)
        if collection_thread and collection_thread.is_alive():
            print("❌ Not enough time between samples — shutting down.")
            stop_event.set()
            break
        for i in range(3): # Collect 3 fractions per sample
            #collection_thread = threading.Thread(target=collect_sample_sim, args=(sample, loc_index, 3))
            collection_thread = threading.Thread(target=collector.collect_fraction, args=(50, loc, loc_index, 20)) #real collection
            collection_thread.start()
            collection_thread.join()
            loc_index += 1

    except queue.Empty:
        pass

    if loc_index >= loc_threshold:  # match number of samples
        break

# Wait for last collection to finish
if collection_thread and collection_thread.is_alive():
    collection_thread.join()

# Final result
if stop_event.is_set() or loc_index < loc_threshold:
    print("❌ Not enough samples")
else:
    print("✅ All samples collected successfully.")

Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']
Rinsing collection tubing for 20 drops...

Current port set to 3
Drop Counter (Drops) Started
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Current port set to 6
Rinsing complete.

Moving to location: well_plate_location at index 0
Movement commands rendered: 4
Moved safely To (X35.5, Y129.5, Z-12):  ['ok']

Current port set to 3
Starting fraction collection at well_plate_location (index 0)...

Drop Counter (Drops) Started
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
Timeout reached.
Failed to collect enough drops at well_plate_location (index 0)...

Current port set to 6
Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']
Rinsing collection tubing for 20 drops...

Current port set to 3
Drop Counte

In [5]:
stop_event.set()